In [1]:
import math
from collections import Counter

# Datensatz definieren
data = [
    {"Tageszeit": "Morgens", "Aktivität": "Arbeiten", "Freundin dabei": "Nein", "Wetter": "Sonnig", "KI-Lernen": "Nein"},
    {"Tageszeit": "Morgens", "Aktivität": "Arbeiten", "Freundin dabei": "Ja", "Wetter": "Regen", "KI-Lernen": "Ja"},
    {"Tageszeit": "Mittags", "Aktivität": "Sport", "Freundin dabei": "Ja", "Wetter": "Sonnig", "KI-Lernen": "Nein"},
    {"Tageszeit": "Mittags", "Aktivität": "Sport", "Freundin dabei": "Nein", "Wetter": "Regen", "KI-Lernen": "Nein"},
    {"Tageszeit": "Abends", "Aktivität": "Programmieren", "Freundin dabei": "Ja", "Wetter": "Sonnig", "KI-Lernen": "Ja"},
    {"Tageszeit": "Abends", "Aktivität": "Programmieren", "Freundin dabei": "Nein", "Wetter": "Regen", "KI-Lernen": "Ja"},
    {"Tageszeit": "Nachmittags", "Aktivität": "Lesen", "Freundin dabei": "Ja", "Wetter": "Sonnig", "KI-Lernen": "Ja"},
    {"Tageszeit": "Nachmittags", "Aktivität": "Lesen", "Freundin dabei": "Nein", "Wetter": "Regen", "KI-Lernen": "Ja"},
    {"Tageszeit": "Morgens", "Aktivität": "Lesen", "Freundin dabei": "Nein", "Wetter": "Sonnig", "KI-Lernen": "Nein"},
    {"Tageszeit": "Mittags", "Aktivität": "Arbeiten", "Freundin dabei": "Ja", "Wetter": "Regen", "KI-Lernen": "Nein"},
    {"Tageszeit": "Abends", "Aktivität": "Sport", "Freundin dabei": "Ja", "Wetter": "Sonnig", "KI-Lernen": "Nein"},
    {"Tageszeit": "Nachmittags", "Aktivität": "Arbeiten", "Freundin dabei": "Nein", "Wetter": "Regen", "KI-Lernen": "Ja"},
    {"Tageszeit": "Morgens", "Aktivität": "Arbeiten", "Freundin dabei": "Nein", "Wetter": "Regen", "KI-Lernen": "Ja"}
]

def entropy(data, target_attr):
    """Berechnet die Entropie einer Liste basierend auf dem Zielattribut."""
    counts = Counter(row[target_attr] for row in data)
    total = len(data)
    values = counts.values()
    single_parts = ((count / total) * math.log2(count / total) for count in values)
    return -sum(single_parts)

def information_gain(data, split_attr, target_attr):
    """Berechnet den Informationsgewinn für ein gegebenes Attribut."""
    total_entropy = entropy(data, target_attr)
    values = set(row[split_attr] for row in data)
    #print('Gesplittete Werte vom Attribut', split_attr, values)
    subset_entropy = 0

    for value in values:
        subset = [row for row in data if row[split_attr] == value]
        weight = len(subset) / len(data)
        subset_entropy += weight * entropy(subset, target_attr)

    return total_entropy - subset_entropy

def id3(data, attributes, target_attr):
    """Rekursive Implementierung des ID3-Algorithmus."""
    # Abbruchbedingung: Wenn alle Zielwerte gleich sind
    target_values = [row[target_attr] for row in data]
    if len(set(target_values)) == 1:
        return target_values[0]

    # Abbruchbedingung: Wenn keine Attribute mehr übrig sind
    if not attributes:
        return Counter(target_values).most_common(1)[0][0]

    # Wähle das Attribut mit dem höchsten Informationsgewinn
    gains = {attr: information_gain(data, attr, target_attr) for attr in attributes}
    print('Information gains:', gains)
    best_attr = max(gains, key=gains.get)

    tree = {best_attr: {}}
    values = set(row[best_attr] for row in data)

    for value in values:
        subset = [row for row in data if row[best_attr] == value]
        subtree = id3(subset, [attr for attr in attributes if attr != best_attr], target_attr)
        tree[best_attr][value] = subtree

    return tree

def print_tree(tree, indent=""):
    """Formatiert die Ausgabe des Entscheidungsbaums."""
    if isinstance(tree, dict):
        for attr, branches in tree.items():
            for value, subtree in branches.items():
                print(f"{indent}{attr} = {value}:")
                print_tree(subtree, indent + "  ")
    else:
        print(f"{indent}--> {tree}")

def evaluate(tree, sample):
    """Evaluieren eines Datensatzes mit dem Entscheidungsbaum."""
    if not isinstance(tree, dict):
        return tree

    attr = next(iter(tree))
    value = sample.get(attr)
    if value not in tree[attr]:
        return None  # Unbekannter Wert
    return evaluate(tree[attr][value], sample)


# ID3-Algorithmus anwenden
target_attr = "KI-Lernen"
attributes = [attr for attr in data[0].keys() if attr != target_attr]
decision_tree = id3(data, attributes, target_attr)

# Entscheidungsbaum ausgeben
print("Entscheidungsbaum:")
print_tree(decision_tree)

# Evaluierung eines Beispieldatensatzes
sample = {"Tageszeit": "Abends", "Aktivität": "Programmieren", "Freundin dabei": "Ja", "Essen": "Pizza"}
result = evaluate(decision_tree, sample)
print("\nEvaluierung des Beispiels:")
print(f"Beispiel: {sample}")
print(f"Ergebnis: {result}")


Information gains: {'Tageszeit': 0.47612072114927406, 'Aktivität': 0.41037049251286306, 'Freundin dabei': 0.0036815326818669947, 'Wetter': 0.10714137637005217}
Information gains: {'Aktivität': 0.31127812445913283, 'Freundin dabei': 0.31127812445913283, 'Wetter': 1.0}
Information gains: {'Aktivität': 0.9182958340544896, 'Freundin dabei': 0.2516291673878229, 'Wetter': 0.2516291673878229}
Entscheidungsbaum:
Tageszeit = Nachmittags:
  --> Ja
Tageszeit = Morgens:
  Wetter = Regen:
    --> Ja
  Wetter = Sonnig:
    --> Nein
Tageszeit = Abends:
  Aktivität = Programmieren:
    --> Ja
  Aktivität = Sport:
    --> Nein
Tageszeit = Mittags:
  --> Nein

Evaluierung des Beispiels:
Beispiel: {'Tageszeit': 'Abends', 'Aktivität': 'Programmieren', 'Freundin dabei': 'Ja', 'Essen': 'Pizza'}
Ergebnis: Ja
